In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import fgbuster
import pysm3
import pysm3.units as u
import pandas as pd
import emcee
import camb
from tqdm import tqdm

from scipy.optimize import minimize
import os

from fgbuster import CMB, Dust, Synchrotron, get_instrument, MixingMatrix

from getdist import MCSamples, plots

import warnings
warnings.filterwarnings('ignore')

import getdist
from getdist import MCSamples, plots

In [2]:
nlmax = 256
ells = np.arange(2,nlmax+1)
coeff = ells*(ells+1)/(2*np.pi)
theory_cl_l = pd.read_csv('../../../cmb_new_ps_l.txt', sep = '\t', header = None, index_col = 0) 

In [3]:
res_fg_a1l64 = np.load('stat_fg/res_a1l64_white.npy')
res_fg_a1l128 = np.load('stat_fg/res_a1l128_white.npy')
res_fg_a1l256 = np.load('stat_fg/res_a1l256_white.npy')
res_fg_a1lv = np.load('stat_fg/res_a1lv_white.npy')

res_fg_avlv = np.load('stat_fg/res_avlv_white.npy')
res_fg_bavlv = np.load('stat_fg/res_bavlv_white.npy')

res_fg_a0l128 = np.load('stat_fg/res_a0l128_white.npy')
res_fg_a1l128 = np.load('stat_fg/res_a1l128_white.npy')
res_fg_a2l128 = np.load('stat_fg/res_a2l128_white.npy')
res_fg_avl128 = np.load('stat_fg/res_avl128_white.npy')

In [4]:
a1l64 = np.load('../results/aps_a1l64_white.npy')
a1l128 = np.load('../results/aps_a1l128_white.npy')
a1l256 = np.load('../results/aps_a1l256_white.npy')
a1lv = np.load('../results/aps_a1lv_white.npy')

a0l128 = np.load('../results/aps_a0l128_white.npy')
a1l128 = np.load('../results/aps_a1l128_white.npy')
a2l128 = np.load('../results/aps_a2l128_white.npy')
avl128 = np.load('../results/aps_avl128_white.npy')

avlv = np.load('../results/aps_avlv_white.npy')
bavlv = np.load('../results/aps_bavlv_white.npy')

In [5]:
pars = camb.set_params(ombh2=0.022, omch2=0.12, tau=0.054,  
                       As=2e-9, ns=0.965, cosmomc_theta = 1.04e-2,
                       r = 1, DoLensing = True, WantTensors = True,
                       Want_CMB_lensing = True, lmax = 1024)

In [6]:
results = camb.get_results(pars)
powers = results.get_cmb_power_spectra(pars, CMB_unit='muK', lmax = 1024)  #returns only the Cls

cl_bb_r1 = powers['tensor'][:,2]

In [7]:
cl_lens = (pd.read_csv('../../../cmb_new_ps_l.txt', sep = '\t', header = None, index_col = 0))
cl_bb = np.array(cl_lens[3])

cl_bb_lens = np.zeros(1025)
cl_bb_lens[2:] = cl_bb[:1023]

In [8]:
ell = np.arange(2, nlmax + 1)
fsky = 1

coeff = ell * (ell + 1)/(2*np.pi)

cl_bb_lens = cl_bb_lens[ell]/coeff
cl_bb_r1 = cl_bb_r1[ell]/coeff

In [9]:
def fisher_matrix(noise, res_fg, r = 0, fsky = 1):
    N = (2*ell+1)*fsky/2
    Cl_tot = cl_bb_r1 * r + cl_bb_lens + noise[ell] + res_fg
    d_Cl_dr = cl_bb_r1

    F_rr = N * (d_Cl_dr/Cl_tot)**2

    return 1/np.sqrt(np.sum(F_rr))

In [10]:
fisher_a0l128 = fisher_matrix(a0l128[1], res_fg_a0l128)
fisher_a1l128 = fisher_matrix(a1l128[1], res_fg_a1l128)
fisher_a2l128 = fisher_matrix(a2l128[1], res_fg_a2l128)
fisher_avl128 = fisher_matrix(avl128[1], res_fg_avl128)

fisher_a1l64 = fisher_matrix(a1l64[1], res_fg_a1l64)
fisher_a1l256 = fisher_matrix(a1l256[1], res_fg_a1l256)
fisher_a1lv = fisher_matrix(a1lv[1], res_fg_a1lv)

fisher_avlv = fisher_matrix(avlv[1], res_fg_avlv)
fisher_bavlv = fisher_matrix(bavlv[1], res_fg_bavlv)

fisher_rr = [fisher_a0l128, fisher_a1l128, fisher_a2l128,
            fisher_a1l64, fisher_a1l256, fisher_a1lv,
            fisher_avl128, fisher_avlv, fisher_bavlv]

In [12]:
labels = [r'$\alpha$ = 0, $\ell_0$ = 128', r'$\alpha$ = -1, $\ell_0$ = 128', r'$\alpha$ = -2, $\ell_0$ = 128',
          r'$\alpha$ = -1, $\ell_0$ = 64', r'$\alpha$ = -1, $\ell_0$ = 256', r'$\alpha$ = -1, $\ell_0$ = v',
          r'$\alpha$ = v, $\ell_0$ = 128', r'$\alpha$ = v, $\ell_0$ = v, Beam', r'$\alpha$ = v, $\ell_0$ = v']
for i in range(len(labels)):
    print(labels[i], '\t F_rr =', np.round(fisher_rr[i], 7))

$\alpha$ = 0, $\ell_0$ = 128 	 F_rr = 3.26e-05
$\alpha$ = -1, $\ell_0$ = 128 	 F_rr = 0.0001175
$\alpha$ = -2, $\ell_0$ = 128 	 F_rr = 0.0003233
$\alpha$ = -1, $\ell_0$ = 64 	 F_rr = 7.82e-05
$\alpha$ = -1, $\ell_0$ = 256 	 F_rr = 0.0001823
$\alpha$ = -1, $\ell_0$ = v 	 F_rr = 0.0001028
$\alpha$ = v, $\ell_0$ = 128 	 F_rr = 0.0003726
$\alpha$ = v, $\ell_0$ = v, Beam 	 F_rr = 0.0003496
$\alpha$ = v, $\ell_0$ = v 	 F_rr = 0.0003511
